# Breast Cancer Detection
## Experiments
This notebook runs an ensemble models.

## Python Packages

In [ ]:
!pip install --upgrade pip --quiet
!pip install wandb --upgrade --quiet
!pip install python-dotenv --quiet
!pip install keras==2.15.0 --quiet
!pip install keras-core --quiet
!pip install tensorflow --quiet

In [5]:
import os
import pathlib
import tensorflow as tf
import pandas as pd
import wandb
import numpy as np
import logging

from base_network import NetworkConfig, Network
# Model Architectures
from aknet import AKNetConfig, AKNetFactory
from mlnet import MLNetConfig, MLNetFactory
from nlnetv2 import NLNetV2Config, NLNetV2Factory
from nlnetv4 import NLNetV4Config, NLNetV4Factory
from nlnetv6 import NLNetV6Config, NLNetV6Factory
from tmnet import TMNetConfig, TMNetFactory
from tmnetv7 import TMNetV7Config, TMNetV7Factory
from zznetv1 import ZZNetV1Config, ZZNetV1Factory
from zznetv2 import ZZNetV2Config, ZZNetV2Factory
from zznetv3 import ZZNetV3Config, ZZNetV3Factory
from ensemble import EnsembleConfig, EnsembleFactory

# Repository controlling persistence of models and experiments
from store import ExperimentRepo

# Base Models
from pretrained import DenseNet, MobileNet, Xception, Inception, VGG, EfficientNet, InceptionResNet, ResNet

# Experiment driver
from experiment import FeatureExtractionExperiment

# Configuration Objects
from config import ProjectConfig, DatasetConfig, CheckPointConfig, TrainConfig, EarlyStopConfig, LearningRateScheduleConfig, Config, ExperimentConfig

# Adapter controls access to secrets, dataset directories, and distribute strategies.
from adapter import KaggleAdapter, Adapter, LocalAdapter

## Parameters

In [6]:
# MAKE SURE DESCRIPTION ACCURATELY REFLECTS THIS RUN.
mode = "Development"
description = "Ensemble"
force = False

## Adapter 
The adapter object encapsulates variables that are platform-dependent, such as device type, distribute strategy, api keys, file paths, etc...

In [7]:
adapter = LocalAdapter(mode=mode)

# Obtain the TensorFlow state and compute distribution policy, i.e. strategy
strategy  = adapter.get_strategy()

# Weights and Biases login for model and metric tracking.
wandb.login(key=adapter.wandb_api_key)

Using CPU
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: aistudio. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/john/.netrc


True

## Reproducibility

In [8]:
def seed_everything():
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
    np.random.seed(hash("improves reproducibility") % 2**32 - 1)
    tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)
seed_everything()

## Build Ensemble Network 

In [9]:
# NLNetV2 DenseNet Model
config = NLNetV2Config(dense1=4096,
                        dense2=2048,
                        dense3=1024)
factory = NLNetV2Factory(config=config)
densenet = factory.create(base_model=DenseNet())

# TMNet EfficientNet Model
config = TMNetConfig(dense1=1024,
                     dense2=1024)
factory = TMNetFactory(config=config)
efficientnet = factory.create(base_model=EfficientNet())

# TMNet Inception Model
config = TMNetConfig(dense1=1024,
                     dense2=1024)
factory = TMNetFactory(config=config)
inception = factory.create(base_model=Inception())

# TMNet Inception/ResNet Model
config = TMNetConfig(dense1=1024,
                     dense2=1024)
factory = TMNetFactory(config=config)
inception_resnet = factory.create(base_model=InceptionResNet())

# TMNet MobileNet Model
config = TMNetConfig(dense1=1024,
                     dense2=1024)
factory = TMNetFactory(config=config)
mobilenet = factory.create(base_model=MobileNet())

# TMNet ResNet Model
config = TMNetConfig(dense1=1024,
                     dense2=1024)
factory = TMNetFactory(config=config)
resnet = factory.create(base_model=ResNet())

# TMNet VGG Model
config = TMNetConfig(dense1=1024,
                     dense2=1024)
factory = TMNetFactory(config=config)
vgg = factory.create(base_model=VGG())

# TMNet Xception Model
config = TMNetConfig(dense1=1024,
                     dense2=1024)
factory = TMNetFactory(config=config)
xception = factory.create(base_model=Xception())

networks = [densenet, efficientnet, inception, inception_resnet, mobilenet, resnet, vgg, xception]

# Create Ensemble
config = EnsembleConfig(dense1=16, dropout1=0.3)
factory = EnsembleFactory(config=config)
network = factory.create(networks=networks)

## Build Experiment Configuration

In [10]:
def build_config(adapter: Adapter, mode: str,network_config: Config, strategy: tf.distribute.Strategy) -> Config:
    """Constructs an experiment Config object """
    # Encapsulates the parameters that define the project in Weights & Biases
    project_config = ProjectConfig(mode=mode)

    # The default batch size is 64; however, if running on TPU, the rule of thumb is to optimally set the batch size to 128 * the number of TPU cores (8).     
    batch_size = 64 if not adapter.device_type == "TPU" else 16 * strategy.num_replicas_in_sync
    dataset_config = DatasetConfig(mode=mode,
                                   batch_size=batch_size)

    # If running on TPU, the learning rate is scaled by the number of cores.
    learning_rate = 1e-3 if not adapter.device_type == "TPU" else 1e-3 * strategy.num_replicas_in_sync
    train_config = TrainConfig(epochs=100, 
                               learning_rate=learning_rate,
                               use_ema=True,
                               momentum=0.9,
                               weight_decay=0.0001,
                               loss="binary_crossentropy",
                               early_stop=True,
                               learning_rate_schedule=True,
                               augmentation=True,
                               checkpoint=False,
                               fine_tune=False,
    )    

    # Checkpoints will be stored in the directory given by the adapter object. 
    checkpoint_config = CheckPointConfig(directory=adapter.model_dir, 
                                         monitor="val_loss", 
                                         verbose=1, 
                                         save_best_only=True, 
                                         save_weights_only=True, 
                                         mode="auto")

    # We'll establish an early stop callback to mitigate overfitting caused by excessive training after validation loss hasn't improved.
    early_stop_config = EarlyStopConfig(min_delta=1e-4, 
                                        monitor="val_loss", 
                                        patience=50, 
                                        restore_best_weights=True, 
                                        verbose=1)
    
    # Learning rate configuration contains the default values for the schedule. 
    learning_rate_schedule_config = LearningRateScheduleConfig(method="reduce_on_plateau",
                                                              min_delta=1e-4,
                                                              min_lr=1e-5,
                                                              monitor="val_loss",
                                                              factor=0.5,
                                                              patience=3,
                                                              restore_best_weights=True,
                                                              verbose=1,
                                                              mode="auto")

    # The experiment configuration is encapsulated into a single object 
    return ExperimentConfig(project=project_config, 
                            dataset=dataset_config, 
                            train=train_config, 
                            network=network_config, 
                            checkpoint=checkpoint_config, 
                            early_stop=early_stop_config,
                            learning_rate_schedule=learning_rate_schedule_config                           
                           )

## Build Dataset

In [11]:

def build_dataset(train_dir: str, subset: str, dataset_config: Config) -> tf.data.Dataset:
    """Produces a TensorFlow training or validation  Dataset  """
    train_dir = pathlib.Path(train_dir).with_suffix('') 
    return tf.keras.utils.image_dataset_from_directory(
        train_dir,
        labels=dataset_config.labels,
        color_mode=dataset_config.color_mode,
        image_size=dataset_config.image_size,
        shuffle=dataset_config.shuffle,
        validation_split=dataset_config.validation_split,
        subset=subset,
        interpolation=dataset_config.interpolation,
        seed=dataset_config.seed,
        batch_size=dataset_config.batch_size
    )

## Dataset Augmentation

In [12]:
def optimize_dataset(train_ds: tf.data.Dataset, augment: bool = True) -> tf.data.Dataset:
    
    hflip = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('horizontal')
    ])
    
    vflip = tf.keras.Sequential([
        tf.keras.layers.RandomFlip('vertical')
    ])
        
    rotate = tf.keras.Sequential([
        tf.keras.layers.RandomRotation(0.2)
    ])
   
    if augment:
    
        # Horizontal flip the original dataset.
        train_ds2 = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (hflip(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE)) 
        
        # Vertically flip the original dataset.
        train_ds3 = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (vflip(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))         

        # Rotate the original dataset
        train_ds4 = (train_ds
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))            

        # Rotate the horizontally flipped dataset
        train_ds5 = (train_ds2
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))     
        
        # Rotate the vertically flipped dataset
        train_ds6 = (train_ds3
                    .cache()
                    .shuffle(buffer_size=len(train_ds)) 
                    .map(lambda x, y: (rotate(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
                    .prefetch(tf.data.AUTOTUNE))             

        # Concatenate the datasets
        train_ds = train_ds.concatenate(train_ds2)
        train_ds = train_ds.concatenate(train_ds3)
        train_ds = train_ds.concatenate(train_ds4)
        train_ds = train_ds.concatenate(train_ds5)
        train_ds = train_ds.concatenate(train_ds6)
        
    return train_ds

## Build Callbacks
The build_callbacks function creates callbacks common to all networks. Early stop is one such callback that is optionally created based upon the train config.

The add_learning_rate_callback adds a network-specific learning rate schedule. In this case, the Triangle learning rate policy is created for each network, based upon the minimum and maximum learning rates obtained by the learning rate range test of each network.

In [13]:
def build_callbacks(config: Config) -> list:
    """Construct an early stop, learning rate, and model checkpoint callback. """    
    
    callbacks = []
    
    if config.train.early_stop:
    
        early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor=config.early_stop.monitor, 
                                                            min_delta=config.early_stop.min_delta,
                                                            patience=config.early_stop.patience, 
                                                            restore_best_weights=config.early_stop.restore_best_weights,
                                                            verbose=config.early_stop.verbose)
        callbacks.append(early_stop_callback)
        
    if config.train.learning_rate_schedule:
        
        if config.learning_rate_schedule.method == "reduce_on_plateau":

            lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor=config.learning_rate_schedule.monitor,
                                                                    factor=config.learning_rate_schedule.factor,
                                                                    patience=config.learning_rate_schedule.patience,
                                                                    verbose=config.learning_rate_schedule.verbose,
                                                                    mode=config.learning_rate_schedule.mode,
                                                                    min_delta=config.learning_rate_schedule.min_delta,
                                                                    min_lr=config.learning_rate_schedule.min_lr)
            callbacks.append(lr_callback)

    return callbacks

## Dependencies

Several dependencies are instantiated / declared:

1. The model factory exposes a create method that constructs a model for the factory's architecture and a pretrained base model. 
2. The repository controls the ways in which models are persisted on Kaggle and Weights & Biases.
3. The optimizer is the algorithm that tunes the weights of the model to minimize the loss function.
4. Designate the metrics to compute during training with in the TensorFlow strategy context.

In [14]:
with strategy.scope():
    metrics = ['accuracy', tf.keras.metrics.AUC()]

## Build and Run Experiments

For each pretrained base model, the following steps are executed:

1. Instantiate a TensorFlow strategy scope context.
2. Create a network object based on the specified base/pretrained model. 
3. Add a network-specific learning rate schedule to the callbacks.
4. Designate tags used for search purposes on Weights & Biases
3. Construct an experiment object for the network
4. Run the experiment.

In [16]:
with strategy.scope():  
    # Build configuration object
    config = build_config(adapter=adapter,mode=mode, network_config=network.config, strategy=strategy)    

    # Extract W&B note from network config.
    notes = f"{description} {config.network.description}"

    # Repository controls persistence of experiment, runs, and models
    repo = ExperimentRepo(mode=mode, project=config.project.name, adapter=adapter)

    # Early stop and learning rate callbacks 
    callbacks = build_callbacks(config=config)

    # Construct datasets and optimize and preprocess training set
    train_ds = build_dataset(train_dir=adapter.train_dir, subset="training", dataset_config=config.dataset)
    train_ds = optimize_dataset(train_ds=train_ds, augment=config.train.augmentation)    
    val_ds = build_dataset(train_dir=adapter.train_dir, subset="validation", dataset_config=config.dataset)

    # Instantiate the optimizer with the current configuration    
    optimizer = tf.keras.optimizers.Adam(learning_rate=config.train.learning_rate)                

    # Tags allowing models and runs to be searched on Weights and Biases
    tags = [network.name for network in networks]
    # The Experiment object compiles and fits the model.
    experiment = FeatureExtractionExperiment(
        network=network, 
        config=config, 
        optimizer=optimizer, 
        repo=repo, 
        metrics=metrics, 
        callbacks=callbacks, tags=tags, notes=notes, force=force)
    experiment.run(train_ds=train_ds, val_ds=val_ds)

Found 276 files belonging to 2 classes.
Using 221 files for training.
Found 276 files belonging to 2 classes.
Using 55 files for validation.
# ================================================================================================ #
                                              Ensemble                                              
# ------------------------------------------------------------------------------------------------ #
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Ensemble_input_layer (InputLay  [(None, 224, 224, 3  0          []                               
 er)                            )]                                                                
                                                                                                  
 model (Functional)             (None, 1)   

Epoch 1/100
24/24 [==============================] - 1284s 50s/step - loss: 0.6906 - accuracy: 0.6116 - auc: 0.5282 - val_loss: 0.6912 - val_accuracy: 0.5636 - val_auc: 0.4913 - lr: 0.0010
Epoch 2/100
24/24 [==============================] - 1045s 43s/step - loss: 0.6859 - accuracy: 0.6139 - auc: 0.5137 - val_loss: 0.6882 - val_accuracy: 0.5636 - val_auc: 0.4724 - lr: 0.0010
Epoch 3/100
24/24 [==============================] - 1022s 42s/step - loss: 0.6810 - accuracy: 0.6169 - auc: 0.4883 - val_loss: 0.6862 - val_accuracy: 0.5636 - val_auc: 0.5000 - lr: 0.0010
Epoch 4/100
24/24 [==============================] - 1037s 43s/step - loss: 0.6758 - accuracy: 0.6154 - auc: 0.5058 - val_loss: 0.6852 - val_accuracy: 0.5636 - val_auc: 0.5000 - lr: 0.0010
Epoch 5/100
24/24 [==============================] - 1033s 43s/step - loss: 0.6736 - accuracy: 0.6154 - auc: 0.4901 - val_loss: 0.6843 - val_accuracy: 0.5636 - val_auc: 0.6277 - lr: 0.0010
Epoch 6/100
24/24 [==============================] - 10